In [ ]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
#import tensorflow as tf
import tensorflow.compat.v1 as tf
import zipfile
import pathlib
import os

from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

if StrictVersion(tf.__version__) < StrictVersion('1.9.0'):
    raise ImportError('Please upgrade your TensorFlow installation to v1.9.* or later!')

In [ ]:
from utils import label_map_util

from utils import visualization_utils as vis_util

In [ ]:
#run this to avoid downloading every time the model
MODEL_NAME = 'ssd_mobilenet_v1_coco_2017_11_17'
PATH_TO_FROZEN_GRAPH = MODEL_NAME + '/frozen_inference_graph.pb'
PATH_TO_LABELS = 'data/mscoco_label_map.pbtxt'

In [ ]:
#--EVEN IF THE MODEL IS NOT DOWNLOADED, THIS CODE BOX MUST STILL BE RUN----
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.io.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

In [ ]:
#the error "tensorflow does not have module gfile" was solved by replacing everywhere with tf.io.gfile.GFile()
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [ ]:
def run_inference_for_single_image(image, graph):
    if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
    image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

    # Run inference
    output_dict = sess.run(tensor_dict,
                            feed_dict={image_tensor: np.expand_dims(image, 0)})

    # all outputs are float32 numpy arrays, so convert types as appropriate
    output_dict['num_detections'] = int(output_dict['num_detections'][0])
    output_dict['detection_classes'] = output_dict[
        'detection_classes'][0].astype(np.uint8)
    output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
    output_dict['detection_scores'] = output_dict['detection_scores'][0]
    if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
    return output_dict

In [ ]:
#importing thr functions for lane detection
#loading the image
def process_image(image):
	height = image.shape[0]
	width = image.shape[1]


	#defining the region of interest
	region_of_interest = [
		(0, 290),
		(330, 210), 
		(550, height)
	]	

	gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
	canny_image = cv2.Canny(image, 100, 200)

	#crop the image leaving only the region of interest
	cropped_image = region_of_interest_mask(canny_image, np.array([region_of_interest], np.int32))

	#now we have to draw the lines
	lines = cv2.HoughLinesP(cropped_image, rho = 6, 
		theta = np.pi/60, threshold = 170, lines = np.array([]), 
		minLineLength = 40, maxLineGap = 25)

	image_with_lines = draw_the_lines(image, lines)
	return image_with_lines


#now i want to cover all areas in the image that are not the region of interest
def region_of_interest_mask(img, vertices):
	mask = np.zeros_like(img)
	#channel_count = img.shape[2]
	match_mask_color = 255
	cv2.fillPoly(mask, vertices, match_mask_color)
	masked_image = cv2.bitwise_and(img, mask)
	return masked_image

#function to draw the lines on the image
def draw_the_lines(img, lines):
	img_copy = np.copy(img)
	blank_image = np.zeros((img.shape[0], img.shape[1], 3), dtype = np.uint8)
	for line in lines:
		for x1, y1, x2, y2 in line:
			cv2.line(blank_image, (x1, y1), (x2, y2), (0, 255, 0), thickness = 1)
	#now i merge the two images
	img = cv2.addWeighted(img, 0.8, blank_image, 1, 0.0)
	return img


In [ ]:
import cv2
cap = cv2.VideoCapture('/Users/pietromanfredi/Desktop/lane_detection/car_on_road.mp4')
try:
    with detection_graph.as_default():
        with tf.Session() as sess:
                # Get handles to input and output tensors
                ops = tf.get_default_graph().get_operations()
                all_tensor_names = {output.name for op in ops for output in op.outputs}
                tensor_dict = {}
                for key in [
                  'num_detections', 'detection_boxes', 'detection_scores',
                  'detection_classes', 'detection_masks'
                ]:
                    tensor_name = key + ':0'
                    if tensor_name in all_tensor_names:
                        tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
                      tensor_name)

                while True:
                    ret, image_np = cap.read()
                    # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
                    image_np_expanded = np.expand_dims(image_np, axis=0) #this code is totally useless
                    b = cv2.resize(image_np,(660,330),fx=0,fy=0, interpolation = cv2.INTER_CUBIC) #it is important to resize
                    # Actual detection.
                    output_dict = run_inference_for_single_image(b, detection_graph)
                    # Visualization of the results of a detection.
                    vis_util.visualize_boxes_and_labels_on_image_array(
                        b,
                        output_dict['detection_boxes'],
                        output_dict['detection_classes'],
                        output_dict['detection_scores'],
                        category_index,
                        instance_masks=output_dict.get('detection_masks'),
                        use_normalized_coordinates=True,
                        line_thickness=8)
                    b = process_image(b) #this is the part that does lane detection
                    cv2.imshow('Object detection',b)
                    if cv2.waitKey(25) & 0xFF == ord('q'):
                        cap.release()
                        cv2.destroyAllWindows()
                        break
except Exception as e:
    print(e)
    cap.release()
    cv2.destroyAllWindows()